In [ ]:
# author: Jana Lasser & Almog Simchon

In [1]:
import pandas as pd
import numpy as np
from os.path import join
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import matplotlib.pyplot as plt

# OLS regression articles

## Regression table functionality

In [2]:
table_header = '''
\\begin{table}[]
    \\footnotesize
    \\centering
    \\begin{tabular}{l|c|c|c|c|c|c}
            & coef. & std. err. & $t$ & $P>\\vert t \\vert$ & [0.025 & 0.975] \\\\
            \\toprule
'''
table_footer = '''
            \\bottomrule
    \end{tabular}
    \caption{TODO}
    \label{tab:TODO}
\end{table}
'''

def make_regression_table(res, data, dep_col, name_map, decimals=3):
    tab = table_header
    conf_int = res.conf_int()
    for var in conf_int.index:
        varname = name_map[var]
        coef = res.params[var]
        stderr = res.bse[var]
        t = res.tvalues[var]
        pval = res.pvalues[var]
        #if pval == 0: pval = "$< 0.001$"
        ci_low = conf_int.loc[var][0]
        ci_hi = conf_int.loc[var][1]
        line = f"\t\t{varname} & {coef:1.3f} & {stderr:1.3f} & {t:1.3f} & {pval:1.3f} & {ci_low:1.3f} & {ci_hi:1.3f} \\\\ \n"
        tab += line
        
    tab += "\t\t\\bottomrule \n"
    mean_dep_var = data[dep_col].mean()
    std_dep_var = data[dep_col].std()
    line = f"\t\t\multicolumn{{2}}{{l}}{{R-squared}} & \multicolumn{{1}}{{r}}{res.rsquared:1.3f} & "
    line += f"\multicolumn{{2}}{{l}}{{Mean dependent var}} & \multicolumn{{2}}{{r}}{mean_dep_var:1.3f} \\\\ \n"
    tab += line
    line = f"\t\t\multicolumn{{2}}{{l}}{{Adjusted R-squared}} & \multicolumn{{1}}{{r}}{res.rsquared_adj:1.3f} & "
    line += f"\multicolumn{{2}}{{l}}{{S.D. dependent var}} & \multicolumn{{2}}{{r}}{std_dep_var:1.3f} \\\\ \n"
    tab += line
    line = f"\t\t\multicolumn{{2}}{{l}}{{Model MSE}} & \multicolumn{{1}}{{r}}{res.mse_model:1.3f} & "
    line += f"\multicolumn{{2}}{{l}}{{AIC}} & \multicolumn{{2}}{{r}}{res.aic:1.3f} \\\\ \n"
    tab += line
    line = f"\t\t\multicolumn{{2}}{{l}}{{Sum squared resid}} & \multicolumn{{1}}{{r}}{res.ssr:1.3f} & "
    line += f"\multicolumn{{2}}{{l}}{{BIC}} & \multicolumn{{2}}{{r}}{res.bic:1.3f} \\\\ \n"
    tab += line
    line = f"\t\t\multicolumn{{2}}{{l}}{{Log-likelihood}} & \multicolumn{{1}}{{r}}{res.llf:1.3f} & "
    line += f"\multicolumn{{2}}{{l}}{{F-statistic}} & \multicolumn{{2}}{{r}}{res.fvalue:1.3f} \\\\ \n"
    tab += line
    dw = durbin_watson(res.resid)
    line = f"\t\t\multicolumn{{2}}{{l}}{{Durbin-Watson stat}} & \multicolumn{{1}}{{r}}{dw:1.3f} & "
    line += f"\multicolumn{{2}}{{l}}{{Prob(F-statistic)}} & \multicolumn{{2}}{{r}}{res.f_pvalue:1.3f} \\\\ \n"
    tab += line
    tab += table_footer
    
    return tab

## OLS regression NewsGuard score on belief & truth similarity

In [3]:
src = "../../data/articles"
fname = "article_corpus_clean_honesty_component_scores_glove.csv.gzip"
article_honesty_scores = pd.read_csv(join(src, fname), compression="gzip")

# note: comment out the following two lines if you don't have access to NewsGuard
# use the independent scores instead and change the regression formula below
# to use the "accuracy" column from the independent scores data frame
fname = "url_NG_scores.csv.gzip"
article_NG_scores = pd.read_csv(join(src, fname), compression="gzip")

#fname = "url_independent_scores.csv.gzip"
#article_independent_scores = pd.read_csv(join(src, fname), compression="gzip")

In [4]:
articles = pd.merge(
    article_honesty_scores,
    article_NG_scores,
    how="left",
    left_on="url",
    right_on="url"
)
articles = articles[articles["party"].isin(["Democrat", "Republican"])]

In [5]:
# normalize and center the scores
articles["NG_score"] = articles["NG_score"] / 100
articles["avg_belief_score"] = articles["avg_belief_score"] - \
    articles["avg_belief_score"].mean()
articles["avg_truth_score"] = articles["avg_truth_score"] - \
    articles["avg_truth_score"].mean()

In [6]:
mod = smf.ols(formula='NG_score ~ avg_belief_score + avg_truth_score + party + party * avg_belief_score + party * avg_truth_score + party * avg_belief_score * avg_truth_score', data=articles)
res = mod.fit()
print(res.summary2().tables[1])

                                                       Coef.  Std.Err.  \
Intercept                                           0.945163  0.000401   
party[T.Republican]                                -0.101883  0.000628   
avg_belief_score                                   -0.004602  0.007275   
party[T.Republican]:avg_belief_score               -0.319362  0.011443   
avg_truth_score                                     0.017289  0.008043   
party[T.Republican]:avg_truth_score                 0.165697  0.012617   
avg_belief_score:avg_truth_score                    0.005367  0.028549   
party[T.Republican]:avg_belief_score:avg_truth_...  0.015853  0.042187   

                                                              t  \
Intercept                                           2354.307857   
party[T.Republican]                                 -162.324036   
avg_belief_score                                      -0.632588   
party[T.Republican]:avg_belief_score                 -27.908840  

In [7]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               NG_score   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                  0.146
Method:                 Least Squares   F-statistic:                     4302.
Date:                Wed, 30 Nov 2022   Prob (F-statistic):               0.00
Time:                        15:04:08   Log-Likelihood:             1.1806e+05
No. Observations:              176244   AIC:                        -2.361e+05
Df Residuals:                  176236   BIC:                        -2.360e+05
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
========================================================================================================================
                                                           coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------
Intercept                                                0.9452      0.000   2354.308      0.000       0.944       0.946
party[T.Republican]                                     -0.1019      0.001   -162.324      0.000      -0.103      -0.101
avg_belief_score                                        -0.0046      0.007     -0.633      0.527      -0.019       0.010
party[T.Republican]:avg_belief_score                    -0.3194      0.011    -27.909      0.000      -0.342      -0.297
avg_truth_score                                          0.0173      0.008      2.150      0.032       0.002       0.033
party[T.Republican]:avg_truth_score                      0.1657      0.013     13.133      0.000       0.141       0.190
avg_belief_score:avg_truth_score                         0.0054      0.029      0.188      0.851      -0.051       0.061
party[T.Republican]:avg_belief_score:avg_truth_score     0.0159      0.042      0.376      0.707      -0.067       0.099
==============================================================================
Omnibus:                    65675.348   Durbin-Watson:                   1.107
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           348916.134
Skew:                          -1.726   Prob(JB):                         0.00
Kurtosis:                       8.967   Cond. No.                         177.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
name_map = {
    "Intercept":"Intercept",
    "party[T.Republican]":"Republican",
    "avg_belief_score":"$D_\\mathrm{b}$",
    "avg_belief_score:avg_truth_score":"$D_\\mathrm{b}$ \\times $D_\\mathrm{t}$",
    "party[T.Republican]:avg_belief_score":"Republican $\\times$ $D_\\mathrm{b}$",
    "avg_truth_score":"$D_\\mathrm{t}$",
    "party[T.Republican]:avg_truth_score":"Rep. $\\times$ $D_\\mathrm{t}$",
    "party[T.Republican]:avg_belief_score:avg_truth_score":"Rep. $\\times$ $D_\\mathrm{b}$ $\\times$ $D_\\mathrm{t}$"
}

tab = make_regression_table(res, articles, "NG_score", name_map)
dst = "../../tables"
fname = "OLS_table_article_NG_score.txt"
with open(join(dst, fname), "w") as f:
    f.write(tab)

## Prediction

In [9]:
honesty_component_score = np.arange(-1.0, 0.4, 0.01)
N = len(honesty_component_score)
belief_pred_df_dem = pd.DataFrame({
    "avg_belief_score":honesty_component_score,
    "party":["Democrat"] * N,
    "avg_truth_score":[articles["avg_truth_score"].mean()] * N
})
belief_pred_df_rep = pd.DataFrame({
    "avg_belief_score":honesty_component_score,
    "party":["Republican"] * N,
    "avg_truth_score":[articles["avg_truth_score"].mean()] * N
})
truth_pred_df_dem = pd.DataFrame({
    "avg_truth_score":honesty_component_score,
    "party":["Democrat"] * N,
    "avg_belief_score":[articles["avg_belief_score"].mean()] * N
})
truth_pred_df_rep = pd.DataFrame({
    "avg_truth_score":honesty_component_score,
    "party":["Republican"] * N,
    "avg_belief_score":[articles["avg_belief_score"].mean()] * N
})

In [10]:
belief_pred_dem = res.get_prediction(belief_pred_df_dem)
belief_pred_rep = res.get_prediction(belief_pred_df_rep)
truth_pred_dem = res.get_prediction(truth_pred_df_dem)
truth_pred_rep = res.get_prediction(truth_pred_df_rep)

belief_pred_dem = belief_pred_dem.summary_frame(alpha=0.05)
belief_pred_dem["party"] = "Democrat"
belief_pred_dem["honesty_component"] = "belief"
belief_pred_rep = belief_pred_rep.summary_frame(alpha=0.05)
belief_pred_rep["party"] = "Republican"
belief_pred_rep["honesty_component"] = "belief"
truth_pred_dem = truth_pred_dem.summary_frame(alpha=0.05)
truth_pred_dem["party"] = "Democrat"
truth_pred_dem["honesty_component"] = "truth"
truth_pred_rep = truth_pred_rep.summary_frame(alpha=0.05)
truth_pred_rep["party"] = "Republican"
truth_pred_rep["honesty_component"] = "truth"

In [11]:
for df in [belief_pred_dem, belief_pred_rep]:
    df["belief"] = honesty_component_score
    df["truth"] = np.nan
    df["honesty_component"] = "belief"
    df.rename(columns={"mean":"NG", "mean_ci_lower":"ymin", "mean_ci_upper":"ymax"}, inplace=True)
for df in [truth_pred_dem, truth_pred_rep]:
    df["belief"] = np.nan
    df["truth"] = honesty_component_score
    df["honesty_component"] = "truth"
    df.rename(columns={"mean":"NG", "mean_ci_lower":"ymin", "mean_ci_upper":"ymax"}, inplace=True)

In [12]:
dst = "../../data/articles"
fname = "OLS_predictions_articles.csv"
cols = ["NG", "belief", "truth", "ymin", "ymax", "party", "honesty_component"]
predictions = pd.concat([
    belief_pred_dem[cols],
    belief_pred_rep[cols],
    truth_pred_dem[cols],
    truth_pred_rep[cols]
]).reset_index(drop=True)
predictions.to_csv(join(dst, fname), index=False)